Source:

huggingface: https://huggingface.co/HooshvareLab/bert-fa-base-uncased-clf-persiannews

Tutorial:https://towardsdatascience.com/fine-tuning-hugging-face-model-with-custom-dataset-82b8092f5333

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
4/1AY0e-g7Y1pG9mT2qWSR6C1fibVLAn85I5hEjCwVx4fTm4dj-Clr9bX74G9Y
Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/sharif/FineTuning/ipython(guide)')

In [32]:
!pip install transformers==2.8

In [5]:
!pip3 install sentencepiece


     |████████████████████████████████| 1.2MB 7.5MB/s 


In [6]:
!git clone https://huggingface.co/HooshvareLab/bert-fa-base-uncased-clf-persiannews
GIT_LFS_SKIP_SMUDGE=1

fatal: destination path 'bert-fa-base-uncased-clf-persiannews' already exists and is not an empty directory.


In [7]:
import torch
import numpy
import pandas
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoConfig,TFAutoModel,AutoModel
from transformers import BertConfig, BertTokenizer
from transformers import TFBertModel, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features, InputExample
from sklearn.metrics import classification_report

In [8]:
# specify GPU
device = torch.device("cuda")

## Data

In [9]:
limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0)
data = data.dropna().reset_index(drop=True)
X = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,Y , test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', len(X_train) , '\ntest: ', len(X_test) , '\nval: ', len(X_val) ,"\ny_tain:",len(y_train) )

train:  12896 
test:  4299 
val:  4299 
y_tain: 12896


In [37]:
##we would load the tokenizer
# from transformers import BertTokenizer, BertModel
# tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

In [38]:
#example
text = "ما در هوشواره معتقدیم با انتقال صحیح دانش و آگاهی، همه افراد میتوانند از ابزارهای هوشمند استفاده کنند. شعار ما هوش مصنوعی برای همه است."
tokenized=tokenizer.tokenize(X_train[0])
input_ids = tokenizer.convert_tokens_to_ids(tokenized)
print(tokenized)
print(input_ids)


['کتابخانه', 'ترفند', 'ها', 'مفیدی', 'ترکیب', 'درخواست', 'ها', 'css', 'جاوااسکریپت', 'فایل', 'اندازه', 'تصویر', 'تلفیق', 'خودکاری', 'sas', '##s', 'فایل', 'ها', 'coffee', '##sc', '##rip', '##t', 'فشرده', 'g', '##zi', '##p', 'عملکرد', 'ها', 'ذکر', 'سرور', 'کاربر', 'بشوند', 'کتابخانه', 'ی', 'mun', '##ee', 'ارايه', 'ی', 'متد', 'ها', 'کلاس', 'ها', 'مفید', '##ش', 'توسعه', 'دهندگان', 'برنامه', 'نویسان', 'توسعه', 'دهندگان', 'php', 'راحتی', 'کار', 'فایل', 'ها', 'css', 'جاوا', 'اسکریپت', 'بپردازند', 'خواست', 'ها', 'ترکیب', 'پروژه', 'ی', 'نمایند', 'امکانات', 'کتابخانه', 'ی', 'mun', '##ee', 'متد', 'ها', 'کتابخانه', 'ی', 'mun', '##ee', 'توسعه', 'دهندگان', 'قادر', 'عکس', 'تصاویر', 'پروژه', 'ی', 'تغییراتی', 'اعمال', 'نمایند', 'تغییرات', 'سایز', 'رنگ', 'وضوح', 'کتابخانه', 'الگو', 'ها', 'امنیتی', 'فایل', 'ها', 'جاوا', 'اسکریپت', 'css', 'قادر', 'فایل', 'ها', 'محافظت', 'ویرایش', 'اصلاح', 'عکس', 'ها', 'حملات', 'مخرب', 'فایل', 'ها', 'جاوا', 'اسکریپت', 'css', 'اتمام', 'کار', 'تصاویر', 'کتابخانه', 'mun', '##

In [39]:
# encode text
sent_id = tokenizer.batch_encode_plus(X_train[:10], padding=True, return_token_type_ids=False)

In [40]:
sent_id

{'input_ids': [[2, 7314, 14551, 5526, 15774, 4950, 3799, 5526, 68588, 85208, 6618, 4224, 4175, 9740, 70611, 66769, 1192, 6618, 5526, 72039, 30311, 42642, 1166, 10966, 35, 58730, 1209, 3361, 5526, 4546, 10562, 5663, 15779, 7314, 382, 41345, 13126, 2783, 382, 21073, 5526, 5588, 5526, 5558, 1176, 2569, 10032, 2385, 21214, 2569, 10032, 66903, 5406, 2109, 6618, 5526, 68588, 25103, 44304, 8194, 3181, 5526, 4950, 3600, 382, 5631, 4499, 7314, 382, 41345, 13126, 21073, 5526, 7314, 382, 41345, 13126, 2569, 10032, 5060, 3679, 4696, 3600, 382, 8274, 4167, 5631, 4403, 9598, 3852, 10029, 7314, 7026, 5526, 4105, 6618, 5526, 25103, 44304, 68588, 5060, 6618, 5526, 8817, 10077, 2942, 3679, 5526, 5318, 10178, 6618, 5526, 25103, 44304, 68588, 6193, 2109, 4696, 7314, 41345, 13126, 6618, 5526, 2559, 3270, 11300, 5170, 6853, 4846, 21015, 7314, 20297, 32035, 1194, 3931, 6853, 2750, 6618, 5526, 7314, 2044, 30825, 7092, 4037, 6853, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [41]:
sentence_maxlen=128

In [42]:
##Tokenize training and validation sentences:
train_encodings = tokenizer.batch_encode_plus(X_train,
    max_length = sentence_maxlen,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False)

val_encodings = tokenizer.batch_encode_plus(X_val,
    max_length = sentence_maxlen,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False)

test_encodings=tokenizer.batch_encode_plus(X_test,
    max_length = sentence_maxlen,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2137: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [43]:
train_encodings

Output hidden; open in https://colab.research.google.com to view.

In [44]:
import torch
import torch.nn as nn

# for train set
train_seq = torch.tensor(train_encodings['input_ids'])
train_mask = torch.tensor(train_encodings['attention_mask'])
train_y = torch.tensor(y_train)

# for validation set
val_seq = torch.tensor(val_encodings['input_ids'])
val_mask = torch.tensor(val_encodings['attention_mask'])
val_y = torch.tensor(y_val)

# for test set
test_seq = torch.tensor(test_encodings['input_ids'])
test_mask = torch.tensor(test_encodings['attention_mask'])
test_y = torch.tensor(y_test)

In [45]:
train_y[0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])

In [46]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

# wrap tensors
test_data = TensorDataset(test_seq, test_mask, test_y)

# sampler for sampling the data during training
test_sampler = SequentialSampler(test_data)

# dataLoader for validation set
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size=batch_size)

In [47]:
# example

text = ["this is a bert model tutorial", "we will fine-tune a bert model"]

# encode text
sent_id = tokenizer.batch_encode_plus(text, padding=True, return_token_type_ids=False)
print(sent_id)

seq = torch.tensor(sent_id['input_ids'])
mask = torch.tensor(sent_id['attention_mask'])
train_y = torch.tensor([0,1])

transformer_model = AutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

cls_hs=transformer_model(seq,mask)
print(cls_hs)
print(cls_hs[0])
print(cls_hs[1])
print(cls_hs[1].shape)

{'input_ids': [[2, 47701, 11684, 29, 41674, 1166, 45084, 19279, 16043, 80349, 4, 0], [2, 19922, 40200, 70992, 14, 63336, 1167, 29, 41674, 1166, 45084, 4]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-1.8125,  0.1581,  0.8727,  ...,  0.5246,  0.0038, -0.6973],
         [-0.2913, -0.2989, -0.2735,  ..., -0.0069,  0.2791, -0.9139],
         [ 0.3463,  0.7396,  0.5679,  ...,  1.3241,  0.0418,  0.3093],
         ...,
         [-1.1403, -0.0708, -0.5872,  ...,  0.3454, -0.1819, -0.4568],
         [-0.3126, -0.1210,  0.4067,  ...,  0.2649,  0.5835, -0.3233],
         [-1.2195,  0.4703, -0.3364,  ..., -0.0554, -0.2337, -0.5484]],

        [[-1.5032, -0.4642,  0.4964,  ...,  0.8154,  0.1530, -0.2859],
         [ 0.8593, -0.1923, -0.2904,  ...,  0.0486,  0.6341, -0.0399],
         [ 1.1156, -0.8193, -0.0553,  ..., -0.0439,  0.3506,  0.4468],
         ...,
         [-1.179

## Model

In [49]:
transformer_model = AutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

In [50]:
# freeze all the parameters
for param in transformer_model.parameters():
    param.requires_grad = False

In [51]:
len(labels)

78

In [52]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,78)

      #sigmoid activation function
      self.sigmoid = nn.Sigmoid()

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      cls_hs = self.bert(sent_id, attention_mask=mask)
      
      x = self.fc1(cls_hs[1])

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply sigmoid activation
      x = self.sigmoid(x)

      return x

In [53]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(transformer_model)

# push the model to GPU
model = model.to(device)

In [54]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-3)

In [55]:
loss_func =nn.MultiLabelSoftMarginLoss()

In [56]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    
    loss = loss_func(preds, labels)
    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = numpy.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [57]:
# function for evaluating the model
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = loss_func(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = numpy.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [58]:
# number of training epochs
epochs = 10

# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 10


/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


  Batch    50  of    403.
  Batch   100  of    403.
  Batch   150  of    403.
  Batch   200  of    403.
  Batch   250  of    403.
  Batch   300  of    403.
  Batch   350  of    403.
  Batch   400  of    403.

Evaluating...
  Batch    50  of    135.
  Batch   100  of    135.

Training Loss: 0.696
Validation Loss: 0.693

 Epoch 2 / 10
  Batch    50  of    403.
  Batch   100  of    403.
  Batch   150  of    403.
  Batch   200  of    403.
  Batch   250  of    403.
  Batch   300  of    403.
  Batch   350  of    403.
  Batch   400  of    403.

Evaluating...
  Batch    50  of    135.
  Batch   100  of    135.

Training Loss: 0.693
Validation Loss: 0.693

 Epoch 3 / 10
  Batch    50  of    403.
  Batch   100  of    403.
  Batch   150  of    403.
  Batch   200  of    403.
  Batch   250  of    403.
  Batch   300  of    403.
  Batch   350  of    403.
  Batch   400  of    403.

Evaluating...
  Batch    50  of    135.
  Batch   100  of    135.

Training Loss: 0.693
Validation Loss: 0.693

 Epoch 4 

#### Save model

In [60]:
path = 'bert-base-parsbert-uncased.pt'
torch.save(model.state_dict(), path)

Loading saved model:

In [ ]:
# torch.cuda.empty_cache()
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(transformer_model)

# push the model to GPU
model = model.to(device)

In [59]:
#load weights of best model
path = 'bert-base-parsbert-uncased.pt'
model.load_state_dict(torch.load(path))

FileNotFoundError: ignored

After loading model:

In [79]:
y_pred=[]
y_true=[]
for step,batch in enumerate(test_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(test_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)
      # print(preds)
      # print(preds.cpu().numpy())
      preds = preds.cpu().numpy()
      # model's performance
    # preds = numpy.argmax(preds, axis = 1)
    
    measure = numpy.mean(preds[0]) + 4*numpy.sqrt(numpy.var(preds[0]))
    for l in preds:
      temp=[]
      for value in l:
        if value >= measure:
          temp.append(1)
        else:
          temp.append(0)
      y_pred.append(temp)
    y_true.extend(labels.cpu().numpy())
    # print(labels.cpu().numpy()[0], preds[0])
print(classification_report(y_true, y_pred))

  Batch    50  of    135.
  Batch   100  of    135.
              precision    recall  f1-score   support

           0       0.03      0.26      0.05       105
           1       0.05      0.46      0.08       162
           2       0.03      0.38      0.06       140
           3       0.00      0.50      0.01        12
           4       0.01      0.17      0.03        76
           5       0.01      0.39      0.01        18
           6       0.03      0.11      0.04       151
           7       0.00      0.17      0.01        35
           8       0.00      0.14      0.01        28
           9       0.02      0.21      0.03       107
          10       0.03      0.38      0.05       121
          11       0.01      0.23      0.02        60
          12       0.06      0.39      0.11       138
          13       0.00      0.07      0.00        14
          14       0.00      0.06      0.00        18
          15       0.02      0.15      0.04       157
          16       0.02      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [71]:
y_pred[-1][:10]

[0, 1, 1, 1, 0, 0, 0, 0, 0, 1]

In [72]:
l[:10]

array([1.1089925e-09, 3.1846412e-09, 3.2408070e-09, 2.1834037e-09,
       6.6468608e-10, 1.0195572e-09, 3.0647768e-10, 1.5789945e-09,
       1.0513498e-09, 1.9063064e-09], dtype=float32)

In [73]:
y_true[-1][:10]

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

In [74]:
measure

1.5832182009845042e-09

In [77]:
numpy.mean(preds[0]) + 3*numpy.sqrt(numpy.var(preds[0]))

2.9233308551646076e-09